In [99]:
import pandas as pd
import numpy as np
import os
# import nltk
import re
from nltk import word_tokenize,sent_tokenize,pos_tag_sents,pos_tag
import time

## 1. Read data.

In [100]:
DATADIR=os.getenv('DATADIR')
print(DATADIR)
DATAPATH = os.path.join(DATADIR, 'labelled_level2.csv.gz')

/Users/felisialoukou/Documents/govuk-taxonomy-supervised-learning/data


In [101]:
data = pd.read_csv(DATAPATH,compression='gzip')
data.shape

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(173560, 21)

In [102]:
data.head()

,base_path,content_id,description,details,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,...,body,combined_text,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon
0,/student-finance-register-login,e57daef4-5eb5-431c-b0ad-14119ab0355f,your student finance online account - check pa...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance login,...,,student finance login your student finance onl...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
1,/student-finance-calculator,434b6eb5-33c8-4300-aba3-f5ead58600b8,student finance calculator - get a quick estim...,{'introductory_paragraph': '<p>This calculator...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,smartanswers,student finance calculator,...,,student finance calculator student finance cal...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
2,/apply-online-for-student-finance,83155b50-418e-437c-9389-cf0e1302900f,apply online for student finance - online appl...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,apply online for student finance,...,,apply online for student finance apply online ...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
3,/student-finance,d38bafd3-2c46-4be2-b50b-50c2ba7d30ed,student finance - student loans or student gra...,"{'parts': [{'title': 'Overview', 'slug': 'over...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance,...,,student finance student finance - student loan...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
4,/apply-for-student-finance,06017464-d3e2-4a4c-8bef-250eff0de7e4,how to apply for student finance - when to app...,"{'parts': [{'title': 'How to apply', 'slug': '...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance: how to apply,...,,student finance: how to apply how to apply for...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN


### 1.2 UTILS.

In [103]:
data.columns

Index(['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'taxon_id', 'taxon_base_path', 'taxon_name', 'level1taxon',
       'level2taxon', 'level3taxon', 'level4taxon', 'level5taxon'],
      dtype='object')

In [104]:
def build_index(x):
    index_dict = {}
    index_dict['index'] = 0
    for i,elem in enumerate(x):
        index_dict[elem] = i+1
    return index_dict

In [105]:
ind = build_index(data.columns)
print(ind['combined_text'])

13


### 1.3 STRIP

In [106]:
def is_junk(word):
    return len(word) < 2

def is_aux(word):
    m = re.match("\\b[iI]s\\b|\\b[aA](m|re)\\b|\\b[bB](een|e)\\b|\\b[hH](ave|as)\\b|\\b[wW](as|ere|ill|ould)\\b",word)
    return bool(m)

In [107]:
test = "been"
print(is_aux(test))
test2 = ""
print(is_junk(test2))

True
True


In [109]:
def reduce_text(x):
    keep_text =""
    sent_pos = pos_tag(word_tokenize(x))
    for word,tag in sent_pos:
        if not is_junk(word) and (tag.startswith("NN") or (tag.startswith("VB") and 
                                                           not is_aux(word)) or tag.startswith("JJ")):
            keep_text += word + " "
    return keep_text.strip()

In [ ]:
data['reduced_combined'] = data['combined_text'].map(reduce_text)

In [111]:
data['reduced_title'] = data['title'].map(reduce_text)

In [116]:
data['reduced_body'] = data['body'].map(reduce_text)

In [112]:
data['reduced_desc'] = data['description'].map(reduce_text)

In [115]:
OUTPUT= os.path.join(DATADIR, 'labelled_level2_filtered_tit_desc.csv.gz')
data.to_csv(OUTPUT,compression='gzip')

In [114]:
data.head()

,base_path,content_id,description,details,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,...,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon,reduced_title,reduced_desc
0,/student-finance-register-login,e57daef4-5eb5-431c-b0ad-14119ab0355f,your student finance online account - check pa...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance login,...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN,student finance login,student finance online account check payment d...
1,/student-finance-calculator,434b6eb5-33c8-4300-aba3-f5ead58600b8,student finance calculator - get a quick estim...,{'introductory_paragraph': '<p>This calculator...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,smartanswers,student finance calculator,...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN,student finance calculator,student finance calculator get quick estimate ...
2,/apply-online-for-student-finance,83155b50-418e-437c-9389-cf0e1302900f,apply online for student finance - online appl...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,apply online for student finance,...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN,online student finance,online student finance online applications stu...
3,/student-finance,d38bafd3-2c46-4be2-b50b-50c2ba7d30ed,student finance - student loans or student gra...,"{'parts': [{'title': 'Overview', 'slug': 'over...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance,...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN,student finance,student finance student loans student grants t...
4,/apply-for-student-finance,06017464-d3e2-4a4c-8bef-250eff0de7e4,how to apply for student finance - when to app...,"{'parts': [{'title': 'How to apply', 'slug': '...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance: how to apply,...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN,student finance apply,apply student finance apply deadlines proof id...
